In [1]:
#Importing relevant modules
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
from skimage.exposure import equalize_adapthist

%matplotlib inline

In [2]:
##Calibrating camera

file_names = glob.glob('camera_cal\calibration*.jpg')

##Calibrating camera
#Number of corners on chessboard
nx = 9
ny = 6

#List of object points and image points
objpoints = []
imgpoints = []

#Setting up objpoint coordinates 
objpoint = np.zeros((6*9,3), np.float32)
objpoint[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

#Reads image 
for file_name in file_names:
    image = mpimg.imread(file_name)

    #Convert image to gray scale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    #Find chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)

    #Appending image points and object points
    if ret == True:
        imgpoints.append(corners)
        objpoints.append(objpoint)
        
#Getting values for camera calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [3]:
#Undistort camera image

def undistort_image(image, mtx, dist):
    '''Returns an undistorted image'''
    undistort_test_image =cv2.undistort(image, mtx, dist, None, mtx)
    
    return undistort_test_image

#Obtaining combined color/gradient threshold binary image

def color_grad(image, color_thresh = (0,255)):
    '''Returns binary output for saturation color threshold'''
    hls_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    s_channel = hls_image[:,:,2]
    
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= color_thresh[0]) & (s_channel <= color_thresh[1])] = 1

    rgb_image = equalize_adapthist(np.copy(image))
    r_channel = rgb_image[:,:,0]
    g_channel = rgb_image[:,:,1]

    r_binary = np.zeros_like(r_channel)
    r_binary[(r_channel>=0.9) & (r_channel <.95) ] = 1
    
    g_binary = np.zeros_like(g_channel)
    g_binary[ (g_channel >= .8)] = 1

    combo = np.zeros_like(r_binary)
    combo[(r_binary==1)|(g_binary==1)|(s_binary ==1)]=1
    return combo
     
def dir_thresh(image, sobel_kernel=9, thresh=(0,np.pi/2)):
    '''Returns binary output for direction of gradient of undistorted image'''
    gray_test = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    
    sobel_x = cv2.Sobel(gray_test, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobel_y = cv2.Sobel(gray_test, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    abs_gradient = np.arctan2(np.absolute(sobel_y), np.absolute(sobel_x))
    
    binary_output_dir = np.zeros_like(abs_gradient)
    binary_output_dir[(abs_gradient >= thresh[0]) & (abs_gradient <= thresh[1])] = 1

    return binary_output_dir

def abs_mag_thresh(image, sobel_kernel=9, mag_thresh=(0, 255)):
    '''Returns binary output for absolute magnitude treshold of undistorted image'''
    gray_test = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    
    sobel_x = cv2.Sobel(gray_test, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobel_y = cv2.Sobel(gray_test, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    abs_mag = np.sqrt(np.square(sobel_y) + np.square(sobel_x))
    
    mag_sobel_xy = np.sqrt(np.square(sobel_x) + np.square(sobel_y))
    scaled_mag_sobel_xy = np.uint8((255*mag_sobel_xy)/np.max(mag_sobel_xy))
    
    binary_output_mag = np.zeros_like(scaled_mag_sobel_xy)
    binary_output_mag[(scaled_mag_sobel_xy >= mag_thresh[0]) & (scaled_mag_sobel_xy <=mag_thresh[1])]= 1
    
    return binary_output_mag

def combined_binary_output(image,dir_kernel = 15, mag_kernel =15, thresh=(0.7,1.4), mag_thresh=(17,80),color_thresh = (0,255)):
    '''Takes binary thresholds and combines them into one image'''
    dir_binary = dir_thresh(image, sobel_kernel=dir_kernel, thresh= thresh)
    abs_binary = abs_mag_thresh(image, sobel_kernel=mag_kernel, mag_thresh=mag_thresh)
    color_binary = color_grad(image,color_thresh)
    
    combined_binary = np.zeros_like(color_binary)
    combined_binary[(color_binary ==1)|((dir_binary == 1) & (abs_binary == 1))] = 1
    
    return combined_binary

In [4]:
#This section of the code visualizes the perspective transform of a test image

def mask_region(image):
    '''Applying mask to only show region of interest'''
    mask = np.zeros_like(image)
    ignore_mask_color = 255
    
    copy_test = np.copy(image)
    
    #Defining four sided polygon to mask
    vertices = np.array([[(170,700),(1250,692), (725,450), (580,450)]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges = cv2.bitwise_and(copy_test, mask)
    
    return masked_edges

def perspective_transform(image, inv = False):
    '''Performs perspective transform on an image. Source and destination points have been hard coded. Might not work for images from other cameras/position'''
    #Source points

    s_bottom_left = [185,700]
    s_bottom_right = [1200,692]
    s_top_right = [736,446]
    s_top_left = [592,446]
    
    source_points = np.float32([s_bottom_left, s_bottom_right, s_top_right, s_top_left])

    #Destination points
    d_bottom_left = [300,700]
    d_bottom_right = [1000,700]
    d_top_right = [1000,0]
    d_top_left = [300,0]

    destination_points = np.float32([d_bottom_left, d_bottom_right, d_top_right, d_top_left])
    
    #Perspective transform
    if inv == False:
        M = cv2.getPerspectiveTransform(source_points,destination_points)
    elif inv == True:
        M = cv2.getPerspectiveTransform(destination_points,source_points)
   
    copy_image = np.copy(image)
    
    #Determine image size for warping images
    if len(image.shape) == 3:
        img_size = image.shape[:2][::-1]
    elif len(image.shape) ==2:
        img_size = image.shape[::-1]
        
    #Warps image
    warped = cv2.warpPerspective(copy_image,M, img_size,flags=cv2.INTER_LINEAR)
    
    return warped

In [5]:
##This section of the code find left and right lane pixels.

def window_line_search(binary_warped):
    '''Finds lane lines with sliding window search'''
    #Find histogram from bottom half of image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)

    #Find midpoint of histogram and find index where maximum occurs for left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    #Set number of windows
    nwindows = 9

    #Set window height
    window_height = np.int(binary_warped.shape[0]/nwindows)

    #Find x and y position of non zero pixels in image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    #Current positions to update window
    leftx_current = leftx_base
    rightx_current = rightx_base

    #Set width of window
    margin = 100

    #Set minimum pixel detected to recenter window
    minpix = 10

    #Empty lists to recieve left and right pixels
    left_lane_inds = []
    right_lane_inds = []

    #For loop to step through each window

    for window in range(nwindows):
        #Identify windows search boundaries in x and y
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        #Identify the nonzero pixels in x and y within a search window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]

        #Append good left and right indices to lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        #Recenters rectangle to their mean position if pixels found is more than minpix
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
        
    #Concatenate array of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    #Extract left and right lane pixels' positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    return left_lane_inds, right_lane_inds, leftx, lefty, rightx, righty, nonzerox, nonzeroy

def secondary_line_search(binary_warped, left_fit, right_fit):
    '''Searches for lines based on previous detections'''
    #Find x and y indices that are nonzero
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    margin = 100

    # Finds indices within margin of known line
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))

    #Extract position of left and right lane lines
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    return left_lane_inds, right_lane_inds, leftx, lefty, rightx, righty, nonzerox, nonzeroy


In [6]:
def polynomial_fit(lefty, leftx, righty, rightx, binary_warped):
    '''Least square second order polynomial fit'''
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx,2)
    left_line = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_line = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fit, right_fit, left_line, right_line

In [7]:
def car_position(left_line, right_line, binary_warped):
    line_seperation = np.mean(((right_line) + (left_line))/2)
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    
    ym_per_pix = 30/700 
    xm_per_pix = 3.7/720
    
    ##Finding offset of car
    center_offset = line_seperation - binary_warped.shape[-1]//2 
    car_offset = np.absolute(center_offset * xm_per_pix)
    
    ##Finding radius of curvature of lane
    y_eval = np.max(ploty)

    # Fit new polynomials to x,y in world space
    left_line_cr = np.polyfit(ploty*ym_per_pix, left_line*xm_per_pix, 2)
    right_line_cr = np.polyfit(ploty*ym_per_pix, right_line*xm_per_pix, 2)

    # Radius of curvature
    left_curverad = ((1 + (2*left_line_cr[0]*y_eval*ym_per_pix + left_line_cr[1])**2)**1.5) / np.absolute(2*left_line_cr[0])
    right_curverad = ((1 + (2*right_line_cr[0]*y_eval*ym_per_pix + right_line_cr[1])**2)**1.5) / np.absolute(2*right_line_cr[0])
    ave_curverad = (left_curverad + right_curverad)/2
                    
    return car_offset, ave_curverad


def final_image(binary_warped,undistort_test_image, left_line, right_line):
    '''Unwarps predicted lines and draws them onto original image'''
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])

    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_line, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_line, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = perspective_transform(color_warp, inv = True)
    
    # Combine the result with the original image
    result = cv2.addWeighted(undistort_test_image, 1, newwarp, 0.3, 0)
    
    left_line = left_line
    right_line = right_line
    binary_warped = binary_warped
    
    car_offset, ave_curverad = car_position(left_line, right_line, binary_warped)
    
    car_offset = 'Car Offset: ' + '{0:.2f}'.format(car_offset) + 'm'
    ave_curverad = 'Radius of Curvature:' + '{0:.2f}'.format(ave_curverad) + 'm'
    
    #Draing car offset and radius of curvature on image
    cv2.putText(result, car_offset , (100, 90), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,0,0), thickness=2)
    cv2.putText(result, ave_curverad , (100, 150), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,0,0), thickness=2)
    
    return result

In [8]:
#Analyzing video

def run_pipeline(image):
    '''Finding lane lines from video'''
    ##Prepocess image
    undistort_test_image = undistort_image(image, mtx, dist)
    combined_binary = combined_binary_output(undistort_test_image, dir_kernel=15, mag_kernel=15, thresh =(0.5,np.pi/2), mag_thresh = (78,200), color_thresh = (175,240))
    masked_image = mask_region(combined_binary)
    binary_warped = perspective_transform(masked_image)
    

    #Performs window line search
    left_lane_inds, right_lane_inds, leftx, lefty, rightx, righty, nonzerox, nonzeroy = window_line_search(binary_warped)
    
    #Performs polynomial fit
    left_fit, right_fit, left_line, right_line = polynomial_fit(lefty, leftx, righty, rightx, binary_warped)
    
    #Draw line
    drawn_image = final_image(binary_warped,undistort_test_image, left_line, right_line)

    return drawn_image
        
        
 

In [9]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
    
white_output = 'test.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(run_pipeline) 
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

[MoviePy] >>>> Building video test.mp4
[MoviePy] Writing video test.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [19:35<00:00,  1.09it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test.mp4 

Wall time: 19min 36s
